In [8]:
import cv2 as cv
import numpy as np 
import matplotlib.pyplot as plt

img = cv.imread('Data Set\Answer Scripts\Answer Scripts\A\JJ5.jpg', cv.IMREAD_GRAYSCALE)
assert img is not None

scale_percent_w= 25  # Percentage of the original size width
scale_percent_h= 30  # Percentage of the original size height
width = int(img.shape[1] * scale_percent_w / 100)
height = int(img.shape[0] * scale_percent_h/ 100)
dim = (width, height)

# Resize image
resized_image = cv.resize(img, dim, interpolation=cv.INTER_AREA)

imshow = cv.imshow('Image', resized_image)
cv.waitKey(0)
cv.destroyAllWindows()


<>:5: SyntaxWarning: invalid escape sequence '\A'
<>:5: SyntaxWarning: invalid escape sequence '\A'
C:\Users\Raveen\AppData\Local\Temp\ipykernel_21356\3827200248.py:5: SyntaxWarning: invalid escape sequence '\A'
  img = cv.imread('Data Set\Answer Scripts\Answer Scripts\A\JJ5.jpg', cv.IMREAD_GRAYSCALE)
